In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon
import json

import the yelp dataset and zillow boundaries

zillow boundaries can be found at: 

https://public.opendatasoft.com/explore/dataset/zillow-neighborhoods/export/

(download a .geojson)

In [2]:
zillow_boundaries = gpd.read_file('/Volumes/data/zillow-neighborhoods_real.geojson')

In [3]:
json_lines = [json.loads( l.strip() ) for l in open('/Volumes/data/yelp_dataset/yelp_academic_dataset_business.json').readlines() ]

In [4]:
dataset_yelpopen = pd.DataFrame(json_lines)

create points for yelp lattitude and longitudes

In [5]:
dataset_yelpopen['latitude'] = dataset_yelpopen['latitude'].astype(float)
dataset_yelpopen['longitude'] = dataset_yelpopen['longitude'].astype(float)

dataset_yelpopen['geometry'] = list(zip(dataset_yelpopen.longitude, dataset_yelpopen.latitude))
dataset_yelpopen['geometry'] = dataset_yelpopen['geometry'].apply(lambda x: Point(x))

## merge with the zillow neighborhood boundaries

convert to 'Geo DataFrame'

In [6]:
zillow_geo = gpd.GeoDataFrame(zillow_boundaries[['regionid','geometry']])
yelp_geo = gpd.GeoDataFrame(dataset_yelpopen[['business_id','geometry']])

In [7]:
zillow_geo.dropna(inplace=True)

In [8]:
yelp_geo.head()

,business_id,geometry
0,Apn5Q_b6Nz61Tq4XzPdf9A,POINT (-114.031674872 51.0918130155)
1,AjEbIBw6ZFfln7ePHha9PA,POINT (-114.939821 35.9607337)
2,O8S5hYJ1SMc8fA4QBtVujA,POINT (-73.5993003 45.5405031)
3,bFzdJJ3wp3PZssNEsyU23g,POINT (-112.0769793 33.4499993)
4,8USyCYqpScwiNEb58Bt6CA,POINT (-114.0273656 51.0355914)


In [9]:
zillow_geo.head()

,regionid,geometry
0,138129,"POLYGON ((-87.78618999999989 41.94558, -87.786..."
1,140779,POLYGON ((-87.65849448899995 40.12343103000006...
2,195637,"POLYGON ((-89.700805 39.812098, -89.696901 39...."
3,269571,"POLYGON ((-87.6337397026425 41.72996999541133,..."
4,269608,"POLYGON ((-87.56564699999987 41.774134, -87.56..."


In [10]:
merged_geos = gpd.sjoin(yelp_geo, zillow_geo, op='within',how='left').fillna('NoNeighborhood')

In [11]:
merged_df = pd.DataFrame(merged_geos[['business_id','regionid']])

In [12]:
merged_df.shape

(188597, 2)

In [13]:
dataset_yelpopen.shape

(188593, 16)

In [14]:
dataset_yelpopen = dataset_yelpopen.merge(merged_df,on='business_id',how='left')

In [16]:
dataset_yelpopen.head(2)

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,neighborhood,postal_code,review_count,stars,state,geometry,regionid
0,1314 44 Avenue NE,"{'BikeParking': 'False', 'BusinessAcceptsCredi...",Apn5Q_b6Nz61Tq4XzPdf9A,"Tours, Breweries, Pizza, Restaurants, Food, Ho...",Calgary,"{'Monday': '8:30-17:0', 'Tuesday': '11:0-21:0'...",1,51.091813,-114.031675,Minhas Micro Brewery,,T2E 6L6,24,4.0,AB,POINT (-114.031674872 51.0918130155),NoNeighborhood
1,,"{'Alcohol': 'none', 'BikeParking': 'False', 'B...",AjEbIBw6ZFfln7ePHha9PA,"Chicken Wings, Burgers, Caterers, Street Vendo...",Henderson,"{'Friday': '17:0-23:0', 'Saturday': '17:0-23:0...",0,35.960734,-114.939821,CK'S BBQ & Catering,,89002,3,4.5,NV,POINT (-114.939821 35.9607337),NoNeighborhood


In [18]:
zillow_boundaries.head()

,city,name,regionid,county,state,geometry
0,Chicago,Dunning,138129,Cook,IL,"POLYGON ((-87.78618999999989 41.94558, -87.786..."
1,Danville,Vermilipn Heights,140779,Vermilion,IL,POLYGON ((-87.65849448899995 40.12343103000006...
2,Springfield,Jefferson Park,195637,Sangamon,IL,"POLYGON ((-89.700805 39.812098, -89.696901 39...."
3,Chicago,Gresham,269571,Cook,IL,"POLYGON ((-87.6337397026425 41.72996999541133,..."
4,Chicago,South Shore,269608,Cook,IL,"POLYGON ((-87.56564699999987 41.774134, -87.56..."


Now this can be merged with zillow stats

https://www.zillow.com/research/data/

For this example, a csv of Rental Values -> Data Type: Price-to-Rent-Ratio -> Geography: Neighborhood

In [19]:
zillow_stats = pd.read_csv('/Volumes/data/Neighborhood_PriceToRentRatio_AllHomes.csv')

In [22]:
zillow_stats_zip = pd.read_csv('/Volumes/data/Zip_PriceToRentRatio_AllHomes.csv', encoding='cp1252')

In [26]:
tidy_stats = zillow_stats[['RegionID','2018-11']]

In [29]:
tidy_stats.columns = ['regionid','PriceToRent']

In [ ]:
tidy_stats.info()

In [32]:
tidy_stats['regionid'] = tidy_stats['regionid'].astype(str)

/Users/debrastern/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


Adding the dollar sign info

In [38]:
dataset_yelpopen['att_string'] = dataset_yelpopen['attributes'].astype(str)
dataset_yelpopen = dataset_yelpopen[dataset_yelpopen['att_string'].str.contains("RestaurantsPriceRange2")]

dollar_signs = []

for each in dataset_yelpopen['attributes']:
    dol = each['RestaurantsPriceRange2']
    dollar_signs.append(dol)

dataset_yelpopen['dollar_sign'] = dollar_signs

/Users/debrastern/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [41]:
dataset_yelpopen['dollar_sign'] = dataset_yelpopen['dollar_sign'].astype(int)

In [46]:
dataset_yelpopen

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,neighborhood,postal_code,review_count,stars,state,geometry,regionid,att_string,dollar_sign
0,1314 44 Avenue NE,"{'BikeParking': 'False', 'BusinessAcceptsCredi...",Apn5Q_b6Nz61Tq4XzPdf9A,"Tours, Breweries, Pizza, Restaurants, Food, Ho...",Calgary,"{'Monday': '8:30-17:0', 'Tuesday': '11:0-21:0'...",1,51.091813,-114.031675,Minhas Micro Brewery,,T2E 6L6,24,4.0,AB,POINT (-114.031674872 51.0918130155),NoNeighborhood,"{'BikeParking': 'False', 'BusinessAcceptsCredi...",2
1,,"{'Alcohol': 'none', 'BikeParking': 'False', 'B...",AjEbIBw6ZFfln7ePHha9PA,"Chicken Wings, Burgers, Caterers, Street Vendo...",Henderson,"{'Friday': '17:0-23:0', 'Saturday': '17:0-23:0...",0,35.960734,-114.939821,CK'S BBQ & Catering,,89002,3,4.5,NV,POINT (-114.939821 35.9607337),NoNeighborhood,"{'Alcohol': 'none', 'BikeParking': 'False', 'B...",2
2,1335 rue Beaubien E,"{'Alcohol': 'beer_and_wine', 'Ambience': '{'ro...",O8S5hYJ1SMc8fA4QBtVujA,"Breakfast & Brunch, Restaurants, French, Sandw...",Montréal,"{'Monday': '10:0-22:0', 'Tuesday': '10:0-22:0'...",0,45.540503,-73.599300,La Bastringue,Rosemont-La Petite-Patrie,H2G 1K7,5,4.0,QC,POINT (-73.5993003 45.5405031),NoNeighborhood,"{'Alcohol': 'beer_and_wine', 'Ambience': ""{'ro...",2
5,"20235 N Cave Creek Rd, Ste 1115","{'BikeParking': 'True', 'BusinessAcceptsCredit...",45bWSZtniwPRiqlivpS8Og,"Coffee & Tea, Food",Phoenix,"{'Monday': '5:30-20:0', 'Tuesday': '5:30-20:0'...",1,33.671375,-112.030017,The Coffee Bean & Tea Leaf,,85024,63,4.0,AZ,POINT (-112.0300171 33.6713751),112547,"{'BikeParking': 'True', 'BusinessAcceptsCredit...",1
6,631 Bloor St W,"{'BusinessParking': '{'garage': False, 'street...",9A2quhZLyWk0akUetBd8hQ,"Food, Bakeries",Toronto,None,0,43.664378,-79.414424,Bnc Cake House,Koreatown,M6G 1K8,7,4.0,ON,POINT (-79.41442379999999 43.6643776),NoNeighborhood,"{'BusinessParking': ""{'garage': False, 'street...",1
7,"3417 Derry Road E, Unit 103","{'Alcohol': 'none', 'BusinessAcceptsCreditCard...",6OuOZAok8ikONMS_T3EzXg,"Restaurants, Thai",Mississauga,None,1,43.712946,-79.632763,Thai One On,Ridgewood,L4T 1A8,7,2.0,ON,POINT (-79.63276310000001 43.7129464),NoNeighborhood,"{'Alcohol': 'none', 'BusinessAcceptsCreditCard...",2
8,1440 N. Dysart Ave,"{'Alcohol': 'none', 'Ambience': '{'romantic': ...",8-NRKkPY1UiFXW20WXKiXg,"Mexican, Restaurants",Avondale,"{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W...",1,33.448106,-112.341302,Filiberto's Mexican Food,,85323,40,2.5,AZ,POINT (-112.341302074 33.4481059352),NoNeighborhood,"{'Alcohol': 'none', 'Ambience': ""{'romantic': ...",1
9,209 Oakland Ave,"{'BikeParking': 'True', 'BusinessAcceptsCredit...",UTm5QZThPQlT35mkAcGOjg,"Flowers & Gifts, Gift Shops, Shopping",Pittsburgh,"{'Monday': '9:0-18:0', 'Tuesday': '9:0-18:0', ...",1,40.441421,-79.956457,Maggie & Stella's Gifts,Oakland,15213,3,3.5,PA,POINT (-79.95645709999999 40.4414214),214069,"{'BikeParking': 'True', 'BusinessAcceptsCredit...",2
11,595 Markham Street,"{'Alcohol': 'full_bar', 'Ambience': '{'romanti...",tZnSodhPwNr4bzrwJ1CSbw,"Cajun/Creole, Southern, Restaurants",Toronto,"{'Tuesday': '17:0-1:0', 'Wednesday': '17:0-1:0...",0,43.664125,-79.411886,Southern Accent Restaurant,Palmerston,M6G 2L7,146,4.0,ON,POINT (-79.4118861 43.6641249),NoNeighborhood,"{'Alcohol': 'full_bar', 'Ambience': ""{'romanti...",2
12,2801 N 15th Ave,"{'Alcohol': 'full_bar', 'Ambience': '{'romanti...",_c3ixq9jYKxhLUB0czi0ug,"Bars, Sports Bars, Dive Bars, Burgers, Nightli...",Phoenix,"{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'...",1,33.479807,-112.091188,Original Hamburger Works,,85007,277,4.0,AZ,POINT (-112.0911877 33.4798071),267926,"{'Alcohol': 'full_bar', 'Ambience': ""{'romanti...",1


### Getting the 'Ambience'

In [51]:
import ast

In [49]:
def ambiencer(row):
    if 'Ambience' not in row.keys():
        pass
    else:
        items = ast.literal_eval(row['Ambience']).items()
        ambiences = []
        for item in items:
            if item[1] == True:
                ambiences.append(item[0])
        return ambiences

In [52]:
dataset_yelpopen['ambience'] = dataset_yelpopen['attributes'].apply(lambda x: ambiencer(x))

In [54]:
dataset_yelpopen['ambience'].fillna('NoAmbience',inplace=True)

In [55]:
dataset_yelpopen.head()

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,...,postal_code,review_count,stars,state,geometry,regionid,att_string,dollar_sign,attribute_string,ambience
0,1314 44 Avenue NE,"{'BikeParking': 'False', 'BusinessAcceptsCredi...",Apn5Q_b6Nz61Tq4XzPdf9A,"Tours, Breweries, Pizza, Restaurants, Food, Ho...",Calgary,"{'Monday': '8:30-17:0', 'Tuesday': '11:0-21:0'...",1,51.091813,-114.031675,Minhas Micro Brewery,...,T2E 6L6,24,4.0,AB,POINT (-114.031674872 51.0918130155),NoNeighborhood,"{'BikeParking': 'False', 'BusinessAcceptsCredi...",2,"{'BikeParking': 'False', 'BusinessAcceptsCredi...",NoAmbience
1,,"{'Alcohol': 'none', 'BikeParking': 'False', 'B...",AjEbIBw6ZFfln7ePHha9PA,"Chicken Wings, Burgers, Caterers, Street Vendo...",Henderson,"{'Friday': '17:0-23:0', 'Saturday': '17:0-23:0...",0,35.960734,-114.939821,CK'S BBQ & Catering,...,89002,3,4.5,NV,POINT (-114.939821 35.9607337),NoNeighborhood,"{'Alcohol': 'none', 'BikeParking': 'False', 'B...",2,"{'Alcohol': 'none', 'BikeParking': 'False', 'B...",NoAmbience
2,1335 rue Beaubien E,"{'Alcohol': 'beer_and_wine', 'Ambience': '{'ro...",O8S5hYJ1SMc8fA4QBtVujA,"Breakfast & Brunch, Restaurants, French, Sandw...",Montréal,"{'Monday': '10:0-22:0', 'Tuesday': '10:0-22:0'...",0,45.540503,-73.599300,La Bastringue,...,H2G 1K7,5,4.0,QC,POINT (-73.5993003 45.5405031),NoNeighborhood,"{'Alcohol': 'beer_and_wine', 'Ambience': ""{'ro...",2,"{'Alcohol': 'beer_and_wine', 'Ambience': ""{'ro...",[]
5,"20235 N Cave Creek Rd, Ste 1115","{'BikeParking': 'True', 'BusinessAcceptsCredit...",45bWSZtniwPRiqlivpS8Og,"Coffee & Tea, Food",Phoenix,"{'Monday': '5:30-20:0', 'Tuesday': '5:30-20:0'...",1,33.671375,-112.030017,The Coffee Bean & Tea Leaf,...,85024,63,4.0,AZ,POINT (-112.0300171 33.6713751),112547,"{'BikeParking': 'True', 'BusinessAcceptsCredit...",1,"{'BikeParking': 'True', 'BusinessAcceptsCredit...",NoAmbience
6,631 Bloor St W,"{'BusinessParking': '{'garage': False, 'street...",9A2quhZLyWk0akUetBd8hQ,"Food, Bakeries",Toronto,None,0,43.664378,-79.414424,Bnc Cake House,...,M6G 1K8,7,4.0,ON,POINT (-79.41442379999999 43.6643776),NoNeighborhood,"{'BusinessParking': ""{'garage': False, 'street...",1,"{'BusinessParking': ""{'garage': False, 'street...",NoAmbience


In [47]:
pd.pivot_table(data=dataset_yelpopen,columns=['postal_code']).T.sort_values('dollar_sign',ascending=False).head()

,dollar_sign,is_open,latitude,longitude,review_count,stars
postal_code,,,,,,
J8B 1Z4,4.0,1.0,45.962787,-74.124379,5.0,4.5
L6T 5B7,4.0,1.0,43.764052,-79.659365,7.0,1.5
L6V 1J6,4.0,1.0,43.690022,-79.765021,9.0,4.0
M6B 4L9,4.0,1.0,43.698901,-79.458523,5.0,2.0
H3G 1W3,4.0,0.0,45.499328,-73.575476,3.0,4.5


In [67]:
dataset_yelpopen

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,...,postal_code,review_count,stars,state,geometry,regionid,att_string,dollar_sign,attribute_string,ambience
0,1314 44 Avenue NE,"{'BikeParking': 'False', 'BusinessAcceptsCredi...",Apn5Q_b6Nz61Tq4XzPdf9A,"Tours, Breweries, Pizza, Restaurants, Food, Ho...",Calgary,"{'Monday': '8:30-17:0', 'Tuesday': '11:0-21:0'...",1,51.091813,-114.031675,Minhas Micro Brewery,...,T2E 6L6,24,4.0,AB,POINT (-114.031674872 51.0918130155),NoNeighborhood,"{'BikeParking': 'False', 'BusinessAcceptsCredi...",2,"{'BikeParking': 'False', 'BusinessAcceptsCredi...",NoAmbience
1,,"{'Alcohol': 'none', 'BikeParking': 'False', 'B...",AjEbIBw6ZFfln7ePHha9PA,"Chicken Wings, Burgers, Caterers, Street Vendo...",Henderson,"{'Friday': '17:0-23:0', 'Saturday': '17:0-23:0...",0,35.960734,-114.939821,CK'S BBQ & Catering,...,89002,3,4.5,NV,POINT (-114.939821 35.9607337),NoNeighborhood,"{'Alcohol': 'none', 'BikeParking': 'False', 'B...",2,"{'Alcohol': 'none', 'BikeParking': 'False', 'B...",NoAmbience
2,1335 rue Beaubien E,"{'Alcohol': 'beer_and_wine', 'Ambience': '{'ro...",O8S5hYJ1SMc8fA4QBtVujA,"Breakfast & Brunch, Restaurants, French, Sandw...",Montréal,"{'Monday': '10:0-22:0', 'Tuesday': '10:0-22:0'...",0,45.540503,-73.599300,La Bastringue,...,H2G 1K7,5,4.0,QC,POINT (-73.5993003 45.5405031),NoNeighborhood,"{'Alcohol': 'beer_and_wine', 'Ambience': ""{'ro...",2,"{'Alcohol': 'beer_and_wine', 'Ambience': ""{'ro...",[]
5,"20235 N Cave Creek Rd, Ste 1115","{'BikeParking': 'True', 'BusinessAcceptsCredit...",45bWSZtniwPRiqlivpS8Og,"Coffee & Tea, Food",Phoenix,"{'Monday': '5:30-20:0', 'Tuesday': '5:30-20:0'...",1,33.671375,-112.030017,The Coffee Bean & Tea Leaf,...,85024,63,4.0,AZ,POINT (-112.0300171 33.6713751),112547,"{'BikeParking': 'True', 'BusinessAcceptsCredit...",1,"{'BikeParking': 'True', 'BusinessAcceptsCredit...",NoAmbience
6,631 Bloor St W,"{'BusinessParking': '{'garage': False, 'street...",9A2quhZLyWk0akUetBd8hQ,"Food, Bakeries",Toronto,None,0,43.664378,-79.414424,Bnc Cake House,...,M6G 1K8,7,4.0,ON,POINT (-79.41442379999999 43.6643776),NoNeighborhood,"{'BusinessParking': ""{'garage': False, 'street...",1,"{'BusinessParking': ""{'garage': False, 'street...",NoAmbience
7,"3417 Derry Road E, Unit 103","{'Alcohol': 'none', 'BusinessAcceptsCreditCard...",6OuOZAok8ikONMS_T3EzXg,"Restaurants, Thai",Mississauga,None,1,43.712946,-79.632763,Thai One On,...,L4T 1A8,7,2.0,ON,POINT (-79.63276310000001 43.7129464),NoNeighborhood,"{'Alcohol': 'none', 'BusinessAcceptsCreditCard...",2,"{'Alcohol': 'none', 'BusinessAcceptsCreditCard...",NoAmbience
8,1440 N. Dysart Ave,"{'Alcohol': 'none', 'Ambience': '{'romantic': ...",8-NRKkPY1UiFXW20WXKiXg,"Mexican, Restaurants",Avondale,"{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W...",1,33.448106,-112.341302,Filiberto's Mexican Food,...,85323,40,2.5,AZ,POINT (-112.341302074 33.4481059352),NoNeighborhood,"{'Alcohol': 'none', 'Ambience': ""{'romantic': ...",1,"{'Alcohol': 'none', 'Ambience': ""{'romantic': ...",[casual]
9,209 Oakland Ave,"{'BikeParking': 'True', 'BusinessAcceptsCredit...",UTm5QZThPQlT35mkAcGOjg,"Flowers & Gifts, Gift Shops, Shopping",Pittsburgh,"{'Monday': '9:0-18:0', 'Tuesday': '9:0-18:0', ...",1,40.441421,-79.956457,Maggie & Stella's Gifts,...,15213,3,3.5,PA,POINT (-79.95645709999999 40.4414214),214069,"{'BikeParking': 'True', 'BusinessAcceptsCredit...",2,"{'BikeParking': 'True', 'BusinessAcceptsCredit...",NoAmbience
11,595 Markham Street,"{'Alcohol': 'full_bar', 'Ambience': '{'romanti...",tZnSodhPwNr4bzrwJ1CSbw,"Cajun/Creole, Southern, Restaurants",Toronto,"{'Tuesday': '17:0-1:0', 'Wednesday': '17:0-1:0...",0,43.664125,-79.411886,Southern Accent Restaurant,...,M6G 2L7,146,4.0,ON,POINT (-79.4118861 43.6641249),NoNeighborhood,"{'Alcohol': 'full_bar', 'Ambience': ""{'romanti...",2,"{'Alcohol': 'full_bar', 'Ambience': ""{'romanti...",[]
12,2801 N 15th Ave,"{'Alcohol': 'full_bar', 'Ambience': '{'romanti...",_c3ixq9jYKxhLUB0czi0ug,"Bars, Sports Bar

In [68]:
madison = dataset_yelpopen.loc[dataset_yelpopen['city'] == 'Madison']

In [71]:
tidy_madison = madison[['name','dollar_sign','geometry']]

In [74]:
gpd.GeoDataFrame(tidy_madison).to_csv('madison_geo.csv')

In [56]:
dataset_yelpopen.head()

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,...,postal_code,review_count,stars,state,geometry,regionid,att_string,dollar_sign,attribute_string,ambience
0,1314 44 Avenue NE,"{'BikeParking': 'False', 'BusinessAcceptsCredi...",Apn5Q_b6Nz61Tq4XzPdf9A,"Tours, Breweries, Pizza, Restaurants, Food, Ho...",Calgary,"{'Monday': '8:30-17:0', 'Tuesday': '11:0-21:0'...",1,51.091813,-114.031675,Minhas Micro Brewery,...,T2E 6L6,24,4.0,AB,POINT (-114.031674872 51.0918130155),NoNeighborhood,"{'BikeParking': 'False', 'BusinessAcceptsCredi...",2,"{'BikeParking': 'False', 'BusinessAcceptsCredi...",NoAmbience
1,,"{'Alcohol': 'none', 'BikeParking': 'False', 'B...",AjEbIBw6ZFfln7ePHha9PA,"Chicken Wings, Burgers, Caterers, Street Vendo...",Henderson,"{'Friday': '17:0-23:0', 'Saturday': '17:0-23:0...",0,35.960734,-114.939821,CK'S BBQ & Catering,...,89002,3,4.5,NV,POINT (-114.939821 35.9607337),NoNeighborhood,"{'Alcohol': 'none', 'BikeParking': 'False', 'B...",2,"{'Alcohol': 'none', 'BikeParking': 'False', 'B...",NoAmbience
2,1335 rue Beaubien E,"{'Alcohol': 'beer_and_wine', 'Ambience': '{'ro...",O8S5hYJ1SMc8fA4QBtVujA,"Breakfast & Brunch, Restaurants, French, Sandw...",Montréal,"{'Monday': '10:0-22:0', 'Tuesday': '10:0-22:0'...",0,45.540503,-73.599300,La Bastringue,...,H2G 1K7,5,4.0,QC,POINT (-73.5993003 45.5405031),NoNeighborhood,"{'Alcohol': 'beer_and_wine', 'Ambience': ""{'ro...",2,"{'Alcohol': 'beer_and_wine', 'Ambience': ""{'ro...",[]
5,"20235 N Cave Creek Rd, Ste 1115","{'BikeParking': 'True', 'BusinessAcceptsCredit...",45bWSZtniwPRiqlivpS8Og,"Coffee & Tea, Food",Phoenix,"{'Monday': '5:30-20:0', 'Tuesday': '5:30-20:0'...",1,33.671375,-112.030017,The Coffee Bean & Tea Leaf,...,85024,63,4.0,AZ,POINT (-112.0300171 33.6713751),112547,"{'BikeParking': 'True', 'BusinessAcceptsCredit...",1,"{'BikeParking': 'True', 'BusinessAcceptsCredit...",NoAmbience
6,631 Bloor St W,"{'BusinessParking': '{'garage': False, 'street...",9A2quhZLyWk0akUetBd8hQ,"Food, Bakeries",Toronto,None,0,43.664378,-79.414424,Bnc Cake House,...,M6G 1K8,7,4.0,ON,POINT (-79.41442379999999 43.6643776),NoNeighborhood,"{'BusinessParking': ""{'garage': False, 'street...",1,"{'BusinessParking': ""{'garage': False, 'street...",NoAmbience
